In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.metrics import precision_recall_fscore_support


In [29]:
features = pd.read_csv("C:/Nini/Capstone/CSV_Files/DataAugmentation_ravdess_siya_extracted_features.csv")
features = features.drop(features.columns[0], axis=1)
temp = shuffle(features)
df = np.random.rand(len(temp)) < 0.8
train = temp[df]
test = temp[~df]

In [30]:
train

,1,2,3,4,5,6,7,8,9,10,...,153,154,155,156,157,158,159,160,161,labels
3230,0.562145,0.578690,0.657932,0.709620,0.749739,0.763275,0.693650,0.636396,0.593934,0.578406,...,0.000059,0.000047,0.000060,0.000076,0.000086,0.000116,7.064165e-05,2.004256e-05,2.813431e-06,Surprised
464,0.510886,0.613796,0.594808,0.529038,0.502751,0.543608,0.594520,0.539021,0.482100,0.465251,...,0.000065,0.000093,0.000056,0.000104,0.000130,0.000155,1.093452e-04,7.537651e-05,1.278255e-05,Happy
1357,0.705760,0.743792,0.734616,0.752404,0.760549,0.683766,0.665228,0.684993,0.691452,0.695329,...,0.000184,0.000170,0.000191,0.000168,0.000189,0.000178,1.592293e-04,1.432693e-04,1.288210e-04,Calm
1418,0.380737,0.355102,0.416155,0.433207,0.458316,0.477618,0.508514,0.446633,0.403362,0.453536,...,0.000009,0.000008,0.000015,0.000011,0.000006,0.000006,6.331102e-06,9.551492e-06,2.226983e-06,Calm
685,0.705340,0.696978,0.675081,0.689953,0.664519,0.645447,0.627494,0.667484,0.688803,0.713105,...,0.000085,0.000083,0.000079,0.000085,0.000092,0.000092,9.353677e-05,7.979224e-05,7.739582e-05,Sad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232,0.599856,0.637036,0.561263,0.523684,0.566721,0.579256,0.625370,0.688927,0.654674,0.632107,...,0.000002,0.000003,0.000003,0.000002,0.000001,0.000001,7.602433e-07,3.848414e-07,5.565179e-08,Calm
4191,0.602304,0.572106,0.518679,0.515817,0.501548,0.507675,0.509237,0.563996,0.555619,0.488843,...,0.003981,0.004607,0.003842,0.002279,0.001404,0.001529,9.635423e-04,8.630424e-04,1.832269e-04,Disgust
1914,0.317125,0.326509,0.295816,0.277044,0.334568,0.536436,0.689249,0.590958,0.421719,0.403533,...,0.002820,0.002540,0.003254,0.003262,0.002763,0.004291,6.532582e-03,5.619395e-03,8.665432e-04,Fearful
2254,0.686048,0.665831,0.670630,0.698302,0.630911,0.605952,0.592821,0.649079,0.677917,0.653152,...,0.009152,0.009772,0.009642,0.010329,0.010877,0.010443,9.100891e-03,8.772907e-03,7.422245e-03,Fearful


In [31]:
X_train = np.array(train.iloc[:,:-1])
y_train = np.array(train.iloc[:,-1])
X_test = np.array(test.iloc[:,:-1])
y_test = np.array(test.iloc[:,-1])

In [32]:
lb = LabelEncoder()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

In [33]:
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.long)

In [34]:
batch_size = 256
train_loader = DataLoader(TensorDataset(X_train,y_train), batch_size=batch_size,shuffle=True)
test_loader = DataLoader(TensorDataset(X_test,y_test), batch_size=batch_size, shuffle=False)

Test 1

In [35]:
class CNNModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=8)
        self.bn1 = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=8)
        self.bn2 = nn.BatchNorm1d(128)
        self.pool1 = nn.MaxPool1d(kernel_size=4, padding=2)
        self.dropout1 = nn.Dropout(0.5)
        self.conv3 = nn.Conv1d(128, 128, kernel_size=8)
        self.bn3 = nn.BatchNorm1d(128)
        self.pool2 = nn.MaxPool1d(kernel_size=4, padding=2)
        self.dropout2 = nn.Dropout(0.5)
        self.conv4 = nn.Conv1d(128, 64, kernel_size=3, padding=2)
        self.bn4 = nn.BatchNorm1d(64)
        self.fc1 = nn.Linear(640, 256)
        self.dropout3 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, num_classes)

    # def _compute_flattened_size(self, input_size):
    #     with torch.no_grad():
    #         x = torch.randn(1, 1, input_size)  # Create a dummy input
    #         x = self.conv1(x)
    #         x = F.relu(x)
    #         x = self.conv2(x)
    #         x = F.relu(x)
    #         x = self.pool1(x)
    #         x = self.dropout1(x)
    #         x = self.conv3(x)
    #         x = F.relu(x)
    #         x = self.pool2(x)
    #         x = self.dropout2(x)
    #         x = self.conv4(x)
    #         x = F.relu(x)
    #         x = x.view(1, -1)  # Flatten
    #         self._to_linear = x.shape[1]

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        x = self.dropout1(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool2(x)
        x = self.dropout2(x)
        x = F.relu(self.bn4(self.conv4(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout3(x)
        x = self.fc2(x)
        return x

Test 2

In [36]:
# class CNNModel(nn.Module):
#     def __init__(self, input_size, num_classes):
#         super(CNNModel, self).__init__()
#         self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=8)
#         self.bn1 = nn.BatchNorm1d(64)
#         self.conv2 = nn.Conv1d(64, 128, kernel_size=8)
#         self.bn2 = nn.BatchNorm1d(128)
#         self.pool1 = nn.MaxPool1d(kernel_size=6, padding=2)
#         self.dropout1 = nn.Dropout(0.5)
#         self.conv3 = nn.Conv1d(128, 128, kernel_size=8)
#         self.bn3 = nn.BatchNorm1d(128)
#         self.pool2 = nn.MaxPool1d(kernel_size=6, padding=2)
#         self.dropout2 = nn.Dropout(0.5)
#         # self.conv4 = nn.Conv1d(128, 64, kernel_size=4, padding=2)
#         # self.bn4 = nn.BatchNorm1d(64)
#         self.fc1 = nn.Linear(384, 256)
#         self.dropout3 = nn.Dropout(0.5)
#         self.fc2 = nn.Linear(256, num_classes)

#     # def _compute_flattened_size(self, input_size):
#     #     with torch.no_grad():
#     #         x = torch.randn(1, 1, input_size)  # Create a dummy input
#     #         x = self.conv1(x)
#     #         x = F.relu(x)
#     #         x = self.conv2(x)
#     #         x = F.relu(x)
#     #         x = self.pool1(x)
#     #         x = self.dropout1(x)
#     #         x = self.conv3(x)
#     #         x = F.relu(x)
#     #         x = self.pool2(x)
#     #         x = self.dropout2(x)
#     #         x = self.conv4(x)
#     #         x = F.relu(x)
#     #         x = x.view(1, -1)  # Flatten
#     #         self._to_linear = x.shape[1]

#     def forward(self, x):
#         x = F.relu(self.bn1(self.conv1(x)))
#         x = F.relu(self.bn2(self.conv2(x)))
#         x = self.pool1(x)
#         x = self.dropout1(x)
#         x = F.relu(self.bn3(self.conv3(x)))
#         x = self.pool2(x)
#         x = self.dropout2(x)
#         # x = F.relu(self.bn4(self.conv4(x)))
#         x = x.view(x.size(0), -1)
#         x = F.relu(self.fc1(x))
#         x = self.dropout3(x)
#         x = self.fc2(x)
#         return x

In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNNModel(input_size=X_train.shape[1], num_classes=8).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)

In [38]:
print("X_train shape:", X_train.shape)

X_train shape: torch.Size([3509, 1, 161])


In [39]:
# epochs = 1500
# for epoch in range(epochs):
#     model.train()
#     running_loss = 0
#     total_loss = 0
#     total = 0
#     correct = 0
#     for X_batch, y_batch in train_loader:
#         X_batch, y_batch = X_batch.to(device), y_batch.to(device)
#         optimizer.zero_grad()
#         outputs = model(X_batch)
#         loss = criterion(outputs, y_batch)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#         _, predicted = torch.max(outputs, 1)  # Get predicted class
#         correct += (predicted == y_batch).sum().item()
#         total += y_batch.size(0)
#     train_accuracy = correct / total
#     avg_train_loss = running_loss / len(train_loader)
#         # total_loss += loss.item()
#         # correct += (outputs.argmax(dim=1) == y_batch).sum().item()
#     model.eval()  # Set model to evaluation mode
#     val_correct = 0
#     val_total = 0
#     val_loss = 0.0

#     with torch.no_grad():  # Disable gradient computation for validation
#         for val_inputs, val_labels in test_loader:
#             val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)

#             val_outputs = model(val_inputs)
#             loss = criterion(val_outputs, val_labels)
#             val_loss += loss.item()

#             _, val_predicted = torch.max(val_outputs, 1)
#             val_correct += (val_predicted == val_labels).sum().item()
#             val_total += val_labels.size(0)
    
#     val_accuracy = val_correct / val_total
#     avg_val_loss = val_loss / len(test_loader)
#     # accuracy = correct / len(train_loader.dataset)
#     print(f"Epoch [{epoch+1}/{epochs}]")
#     print(f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.4f}")
#     print(f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}")
#     print("-" * 50)

With Precision, recall and f1-score

In [40]:
epochs = 1500
for epoch in range(epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    # Training loop
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)  # Get predicted class
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)
    
    train_accuracy = correct / total
    avg_train_loss = running_loss / len(train_loader)

    # Validation loop
    model.eval()
    val_correct = 0
    val_total = 0
    val_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for val_inputs, val_labels in test_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
            val_outputs = model(val_inputs)
            loss = criterion(val_outputs, val_labels)
            val_loss += loss.item()
            _, val_predicted = torch.max(val_outputs, 1)

            val_correct += (val_predicted == val_labels).sum().item()
            val_total += val_labels.size(0)

            # Store predictions and labels for evaluation
            all_preds.extend(val_predicted.cpu().numpy())
            all_labels.extend(val_labels.cpu().numpy())

    val_accuracy = val_correct / val_total
    avg_val_loss = val_loss / len(test_loader)

    # Compute Precision, Recall, F1-score
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted',zero_division=1)

    print(f"Epoch [{epoch+1}/{epochs}]")
    print(f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.4f}")
    print(f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")
    print("-" * 50)

Epoch [1/1500]
Train Loss: 2.0745, Train Acc: 0.1525
Val Loss: 2.0567, Val Acc: 0.1899
Precision: 0.8059, Recall: 0.1899, F1-score: 0.0752
--------------------------------------------------
Epoch [2/1500]
Train Loss: 2.0281, Train Acc: 0.1926
Val Loss: 2.0093, Val Acc: 0.1936
Precision: 0.8062, Recall: 0.1936, F1-score: 0.0763
--------------------------------------------------
Epoch [3/1500]
Train Loss: 1.9871, Train Acc: 0.2052
Val Loss: 1.9540, Val Acc: 0.2158
Precision: 0.7098, Recall: 0.2158, F1-score: 0.1227
--------------------------------------------------
Epoch [4/1500]
Train Loss: 1.9437, Train Acc: 0.2214
Val Loss: 2.0763, Val Acc: 0.1517
Precision: 0.3459, Recall: 0.1517, F1-score: 0.0882
--------------------------------------------------
Epoch [5/1500]
Train Loss: 1.9064, Train Acc: 0.2192
Val Loss: 2.0627, Val Acc: 0.1430
Precision: 0.4530, Recall: 0.1430, F1-score: 0.0681
--------------------------------------------------
Epoch [6/1500]
Train Loss: 1.8682, Train Acc: 0.25

In [41]:
torch.save(model.state_dict(), "C:/Nini/Capstone/Models/DataAugmentation_Noise_pitch_cnn_model.pth")